In [13]:
import pandas as pd
import matplotlib.pyplot as plt

In [14]:
item_df = pd.read_csv("./data/items.csv")
item_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [15]:
item_category_df = pd.read_csv("./data/item_categories.csv")
item_category_df.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [36]:
shop_df = pd.read_csv("./data/shops.csv")
shop_df.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [43]:
dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
sales_df = pd.read_csv("./data/sales_train_v2.csv", parse_dates=['date'], date_parser=dateparse)

In [54]:
sales_df.sort_values(by=["date"], ascending=False).head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2891940,2015-10-31,33,38,5319,299.0,1.0
2892341,2015-10-31,33,37,18045,249.0,1.0
2908046,2015-10-31,33,15,5666,3490.0,1.0
2930914,2015-10-31,33,22,21787,249.0,1.0
2908061,2015-10-31,33,15,5657,5690.0,1.0


In [70]:
import datetime 
date_after = datetime.date(2015, 8, 1)

result = sales_df[sales_df["date"] > date_after].groupby(["shop_id", "item_id"])["item_cnt_day"].sum() / 3
result = result.reset_index()

d:\Miniconda3\envs\ml\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


In [77]:
result.head()

,shop_id,item_id,item_cnt_day
0,2,31,0.333333
1,2,32,0.333333
2,2,33,0.333333
3,2,70,0.333333
4,2,482,0.333333


In [80]:
result["shop_id"] = result["shop_id"].map(str)
result["item_id"] = result["item_id"].map(str)

In [82]:
result['id'] = result[['shop_id', 'item_id']].apply(lambda x: '-'.join(x), axis=1)
result.head()

,shop_id,item_id,item_cnt_day,id
0,2,31,0.333333,2-31
1,2,32,0.333333,2-32
2,2,33,0.333333,2-33
3,2,70,0.333333,2-70
4,2,482,0.333333,2-482


In [35]:
test_df = pd.read_csv("./data/test.csv")
test_df.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [84]:
test_df["shop_id"] = test_df["shop_id"].map(str)
test_df["item_id"] = test_df["item_id"].map(str)

In [85]:
test_df['new_id'] = test_df[['shop_id', 'item_id']].apply(lambda x: '-'.join(x), axis=1)
test_df.head()

,ID,shop_id,item_id,new_id
0,0,5,5037,5-5037
1,1,5,5320,5-5320
2,2,5,5233,5-5233
3,3,5,5232,5-5232
4,4,5,5268,5-5268


In [87]:
submssion_result = pd.merge(test_df, result, how="left", left_on="new_id", right_on="id")

In [88]:
submssion_result.head()

,ID,shop_id_x,item_id_x,new_id,shop_id_y,item_id_y,item_cnt_day,id
0,0,5,5037,5-5037,5,5037,1.333333,5-5037
1,1,5,5320,5-5320,NaN,NaN,NaN,NaN
2,2,5,5233,5-5233,5,5233,1.666667,5-5233
3,3,5,5232,5-5232,5,5232,0.333333,5-5232
4,4,5,5268,5-5268,NaN,NaN,NaN,NaN


In [89]:
del submssion_result["shop_id_x"]
del submssion_result["item_id_x"]
del submssion_result["new_id"]
del submssion_result["shop_id_y"]
del submssion_result["item_id_y"]
del submssion_result["id"]

In [92]:
submssion_result.columns = ['ID', 'item_cnt_month']
submssion_result.head()

,ID,item_cnt_month
0,0,1.333333
1,1,NaN
2,2,1.666667
3,3,0.333333
4,4,NaN


In [94]:
submssion_result.fillna(0, inplace=True)

In [96]:
submssion_result.head()

,ID,item_cnt_month
0,0,1.333333
1,1,0.000000
2,2,1.666667
3,3,0.333333
4,4,0.000000


In [97]:
submssion_result.to_csv('submission.csv', index_col = False)


TypeError: to_csv() got an unexpected keyword argument 'index_col'